In [3]:
# Lozano Garcia Eduardo Alejandro         (Github: Ale9806)
# BMSIS Visiting Scholar,Washington D.C. ,IPN Biomedical Engeneering Mexico City 
# 10/29/2020?

import numpy as np
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import cv2

## PARTE 2

In [4]:
def imread(file):
    """ Reads image """
    path = 'images\\'                            # Set path of images 
    img = cv2.imread(path+file)                  # Read image with opencv
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    return img                   

def imshow(img):
    try:                              # Try  
        img.shape(2)                  # This will only run if we enconunter a tensor of order 3 (n,m,c*)  therefore image is RGB             
        plt.imshow(img)               # Plot RGB image
    except:                           # Except
        gray  =  plt.get_cmap("gray")                   # Select gray cmap
        norm  = mpl.colors.Normalize(vmin=0 ,vmax=255)  # Normalize from 0 to 255
        plt.imshow(img,cmap=gray,norm=norm )            # Show gray image (matrix) with cmpa 
        
    plt.axis('off')                   # Delete axis 
    plt.show()                        # Show image 
    
def imgray(img):
    """ Transforms RGB Image to grayscale image """
    ### weithing average of the 3 channels ###
    img_gray = (0.333*img[:,:,0] + 0.333*img[:,:,1] +  0.333*img[:,:,2]).astype(np.uint8)  
    return img_gray

 
def plot_operation(img,morp,title):
    """ Comapres original image versus morphological transformation"""
    fig, axis = plt.subplots(ncols=2,nrows=1,figsize=(10,5))
    axis[0].imshow(img,"gray") 
    axis[0].set_title("Original Image")       
    axis[0].axis('off')
    axis[1].imshow(morp,"gray")  
    axis[1].set_title(title)    
    axis[1].axis('off')
    plt.show()

In [5]:
def maska(img,Color="RED",Morph=True):
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    if Color == "RED":                          
        Rm,RM = 120,255
        Gm,GM = 0,80
        Bm,BM = 0,80
      

    elif Color == "BLUE":
        Rm,RM = 0,80
        Gm,GM = 0,160
        Bm,BM = 40,255
     
     

    elif Color == "GREEN":
        Rm,RM = 0,80
        Gm,GM = 50,255
        Bm,BM = 0,80
       

    R,G,B = img[:,:,0],img[:,:,1],img[:,:,2]         # Separate img into RGB
    MR = 1*(R <= RM) *  1*(R >= Rm)                  # Red Mask
    MG = 1*(G <= GM) *  1*(G >= Gm)                  # Green Mask
    MB = 1*(B <= BM) *  1*(B >= Bm)                  # Blue Mask

    M = (MR * MG * MB).astype(np.uint8)              # Get total mask
    
    ### Apply Morphological operations ###
    kernel = np.ones((1,1),np.uint8)                 # Initialize kernel
    if Morph:
    
        M = cv2.morphologyEx(M, cv2.MORPH_OPEN, kernel)  # Apply Openning 
        M = cv2.morphologyEx(M, cv2.MORPH_CLOSE, kernel) # Apply Closing 
   
   
    
    return M
    
    
    
    
    
    
def hilight(img,Color="RED",Morph=True,Plot=True):
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    if Color == "RED":                          
        Rm,RM = 120,255
        Gm,GM = 0,80
        Bm,BM = 0,80
        title="Red"

    elif Color == "BLUE":
        Rm,RM = 0,80
        Gm,GM = 0,160
        Bm,BM = 40,255
        title="BLUE"
     

    elif Color == "GREEN":
        Rm,RM = 0,80
        Gm,GM = 50,255
        Bm,BM = 0,80
        title="GREEN"

    R,G,B = img[:,:,0],img[:,:,1],img[:,:,2]         # Separate img into RGB
    MR = 1*(R <= RM) *  1*(R >= Rm)                  # Red Mask
    MG = 1*(G <= GM) *  1*(G >= Gm)                  # Green Mask
    MB = 1*(B <= BM) *  1*(B >= Bm)                  # Blue Mask

    M = (MR * MG * MB).astype(np.uint8)              # Get total mask
    
    ### Apply Morphological operations ###
    kernel = np.ones((1,1),np.uint8)                 # Initialize kernel
    if Morph:
        M = cv2.morphologyEx(M, cv2.MORPH_OPEN, kernel)  # Apply Openning 
        M = cv2.morphologyEx(M, cv2.MORPH_CLOSE, kernel) # Apply Closing 
    img2 = np.copy(img)                              # Copy image so we can modify it
    
    img2[:,:,0] = img[:,:,0]*M                       # Multiply Red   Channel por total Mask
    img2[:,:,1] = img[:,:,1]*M                       # Multiply Green Channel por total Mask
    img2[:,:,2] = img[:,:,2]*M                       # Multiply Blue  Channel por total Mask
 
    new = img - img2                                 # Delet found object to total image
    
    avg = (0.33*new[:,:,0] + 0.33*new[:,:,1] + 0.33*new[:,:,2]).astype(np.uint8)  
    new[:,:,0], new[:,:,1],new[:,:,2]   = avg,avg,avg 
    
    new = new + img2
    
    if Plot:
        plot_operation(img,new,title)
        
    new = cv2.cvtColor(new, cv2.COLOR_RGB2BGR)  # Convert from BGR to RGB
    return new 

In [14]:
capture = cv2.VideoCapture('images\\lala.mp4')  # load video

scale_percent = 40
intilization  =  True
status = True                           # Variable to iterate over frames
while status:                           # While loop 
    
    status,frame = capture.read()       # Reads video, returns (status,frame)
    if status == True:                  # If there is a frame
        if   intilization:                                              # We only get the width and heigth of the first frame since the video is unifrom
                width = int(frame.shape[1] * scale_percent / 100)       # Get width
                height = int(frame.shape[0] * scale_percent / 100)      # Get height
                dim = (width, height) 
                out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, dim) # Create wrtier 
                intilization = False
        
        frame =  cv2.resize(frame, dim, interpolation = cv2.INTER_AREA) # Reszie frame
       
       # frame =  hilight(frame,Color ="BLUE",Morph=True,Plot=False)    # Hilight 1 color 
        MR = maska(frame,Color ="BLUE",Morph=True) 
        MB = maska(frame,Color="RED"  ,Morph=True)
        MT = MR + MB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        img2 = np.copy(frame)                           # Copy image so we can modify it
    
        img2[:,:,0] = frame[:,:,0]*MT                   # Multiply Red   Channel por total Mask
        img2[:,:,1] = frame[:,:,1]*MT                   # Multiply Green Channel por total Mask
        img2[:,:,2] = frame[:,:,2]*MT                   # Multiply Blue  Channel por total Mask

        new = frame - img2                              # Delet found object to total image

        avg = (0.33*new[:,:,0] + 0.33*new[:,:,1] + 0.33*new[:,:,2]).astype(np.uint8)  
        new[:,:,0], new[:,:,1],new[:,:,2]   = avg,avg,avg 

        new = new + img2
        new = cv2.cvtColor(new, cv2.COLOR_RGB2BGR)  # Convert from BGR to RGB
        out.write(new)
        cv2.imshow("Video",new)       # Display it on Window
    


    if cv2.waitKey(1) & 0xFF == ord('q'): # If q is pressed 
        break                             # Break from while loop

cv2.destroyAllWindows()                   # Destroy window
capture.release()                         # Release video (free memory)
out.release()    